In [1]:
import sgkit as sg
import pandas as pd
import numpy as np
import zarr
import xarray as xr
import allel

In [2]:
import glob

ds1 = sg.load_dataset('/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_2L.zarr')
ds1

ds2 = sg.load_dataset('/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_2R.zarr')
ds2

ds3 = sg.load_dataset('/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_3L.zarr')
ds3 

ds4 = sg.load_dataset('/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_3R.zarr')
ds4 

ds5 = sg.load_dataset('/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_X.zarr')
ds5 

<xarray.Dataset>
Dimensions:               (variants: 684159, samples: 18, alleles: 4,
                           ploidy: 2, alt_alleles: 3)
Dimensions without coordinates: variants, samples, alleles, ploidy, alt_alleles
Data variables: (12/15)
    call_AD               (variants, samples, alleles) int32 dask.array<chunksize=(10000, 18, 4), meta=np.ndarray>
    call_DP               (variants, samples) int32 dask.array<chunksize=(10000, 18), meta=np.ndarray>
    call_GQ               (variants, samples) int32 dask.array<chunksize=(10000, 18), meta=np.ndarray>
    call_genotype         (variants, samples, ploidy) int8 dask.array<chunksize=(10000, 18, 2), meta=np.ndarray>
    call_genotype_mask    (variants, samples, ploidy) bool dask.array<chunksize=(10000, 18, 2), meta=np.ndarray>
    call_genotype_phased  (variants, samples) bool dask.array<chunksize=(10000, 18), meta=np.ndarray>
    ...                    ...
    variant_MQ            (variants) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_allele        (variants, alleles) object dask.array<chunksize=(10000, 4), meta=np.ndarray>
    variant_contig        (variants) int8 dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_id            (variants) object dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_id_mask       (variants) bool dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_position      (variants) int32 dask.array<chunksize=(10000,), meta=np.ndarray>
Attributes:
    contigs:               ['AgamP4_2L', 'AgamP4_2R', 'AgamP4_3L', 'AgamP4_3R...
    max_alt_alleles_seen:  4
    source:                sgkit-0.4.0

In [11]:
def thres_no(ds, thres_number):

    ds['sample_cohort'] = xr.DataArray([0] * 2 + [1] * 10 + [2] * 6, dims="samples")
    ds['sample_cohort_name'] = xr.DataArray(['ara'] * 2 + ['gam'] * 10 + ['hyb_ctr'] + ['hyb'] * 5, dims="samples")
    ds['sample_cohort_name']

    sample_is_hyb = ds['sample_cohort_name'] == 'hyb'
    sample_is_ara = ds['sample_cohort_name'] == 'ara'
    sample_is_gam = ds['sample_cohort_name'] == 'gam'
    
    g = allel.GenotypeArray(ds['call_genotype'])

    is_fixed_ara_test=g[:,sample_is_ara].is_hom_alt().all(axis=1)
    g_markers = g[is_fixed_ara_test,:]

    # original array is filtered to contain only ARA samples
    g_hyb_markers = g_markers[:,sample_is_hyb,:]

    is_het_hyb_test = g_hyb_markers.is_het().sum(axis=1) >= thres_number
    return is_het_hyb_test.sum()

In [13]:
num_snps = {}

for contig in ['AgamP4_2L', 'AgamP4_2R', 'AgamP4_3L', 'AgamP4_3R', 'AgamP4_X']:
    print(contig)
    ds = sg.load_dataset(f'/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/{contig}.zarr')
    num_snps[contig] = []
    
    for threshold in [2, 3, 4, 5]:
        print(threshold,)
        n = thres_no(ds, threshold)
        num_snps[contig].append(n)

AgamP4_2L
2
3
4
5
AgamP4_2R
2
3
4
5
AgamP4_3L
2
3
4
5
AgamP4_3R
2
3
4
5
AgamP4_X
2
3
4
5


In [23]:
import pandas as pd

snp_count_df = pd.DataFrame(num_snps).T
snp_count_df.columns = ['Threshold_2', "Threshold_3",'Threshold_4', "Threshold_5" ]
snp_count_df


,Threshold_2,Threshold_3,Threshold_4,Threshold_5
AgamP4_2L,2166,1329,288,10
AgamP4_2R,6363,5188,4611,4184
AgamP4_3L,568,118,56,6
AgamP4_3R,165,106,16,0
AgamP4_X,40,20,13,3


In [ ]:
snp_count_df.to_csv("/Users/christinagkertsou/Desktop/Threshold_table.csv")